### importing require packages

In [2]:
from __future__ import print_function

import json
import os
import numpy as np
import sys

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence

from nltk.tokenize import word_tokenize

### getting data from preprocessing

In [ ]:
word2vec_weights = get_weights()
word2index, index2word = get_vocabulary()
word2vec_model = get_model()
tokenized_indexed_sentences = get_tokenized_indexed_sentences()

### generating training data

In [280]:
window_size = min([len(sentence) for sentence in tokenized_indexed_sentence ])/2
vocab_size = len(word2index)
#sentence_max_length = max([len(sentence) for sentence in tokenized_indexed_sentence ])

Dataset size :  77


In [ ]:
seq_in = []
seq_out = []
# generating dataset
for sentence in tokenized_indexed_sentences:
    for i in range(len(sentence)-window_size-1):
        x = sentence[i:i + window]
        y = sentence[i + window]
        seq_in.append(x)#[]
        seq_out.append(word2vec_weights[y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

## defining model

In [284]:
model = Sequential()
model.add(Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights]))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(100, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 100)         8700      
_________________________________________________________________
lstm_33 (LSTM)               (None, None, 512)         1255424   
_________________________________________________________________
dropout_25 (Dropout)         (None, None, 512)         0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 64)                147712    
_________________________________________________________________
dropout_26 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 100)               6500      
Total params: 1,418,336
Trainable params: 1,418,336
Non-trainable params: 0
_________________________________________________________________


## training model

In [ ]:
model.fit(seq_in, seq_out, batch_size=32, epochs=50, verbose=1)

### model predict

In [287]:
start = 0
pattern = list(seq_in[start])
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(10):
    prediction = model.predict(np.array([pattern]))
    pred_word = w2v_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word+" ")
    pattern.append(word2index[pred_word])
    pattern = pattern[1:len(pattern)]